## Task 1 Downloading and reading data

The first step in analysing any unfamiliar data is to read it into memory and find out some information about its structure.
We will download a mode 357 bst file and then complete/answer the following:
* Read BST file it into memory. (Hint, `numpy` makes things easy)
* How many data points are there?
* What is the bitmode? i.e. How do the number of data points compare to the file size?
* How many subbands were observed? (Hint, need to know bitmode of observation)
* How long was the observation?



In [ ]:
import os

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time, TimeDelta
from matplotlib import dates


%matplotlib notebook
!wget https://data.lofar.ie/2017/09/02/bst/kbt/rcu357_1beam/20170902_103626_bst_00X.dat -P ./data

#Read in data
#bstfile357 = "./data/20170902_103626_bst_00X.dat"
#data = np.fromfile(...)
#os.path.getsize(...)

## Task 2 Reshaping data

BST data is 2 dimensional, time and frequency. Using what we learned above reshape the data so that it is a 2D array with time on one axis and frequency on another.

In [ ]:
#Reshape data
#data = data.reshape(time,frequency)

## Task 3 Quick plot

We now have everything we need to make a quick plot of our data.

* Use `matplotlib` to produce a dynamic spectrum. Make sure to have time on the x-axis.
* What "mode" was this observation made in? (Hint, log your data)
* Play around with vmin and vmax.
* Do all subbands contain data? If not, how many empty subbands are there?

In [ ]:
#plt.imshow(...)

## Task 4 Updating axes

Let's convert subband number to frequency and include the actual time for our axes.

Remember the conversion from subband number to frequency from earlier

$$\nu = (n-1+ \frac{s}{512})\frac{\nu_{clock}}{2},$$

where $n$ is the nyquist zone, $s$ is the subband number and $\nu_{clock}$ is the clock frequency.

In this example $\nu_{clock} = 200$ MHz.

* Write a function to convert subband number to frequency.
* Using the timestamp in the BST file name, create an array of datetime objects for the time array.
* Replot the data with the correct frequency and time axis.


Remember that the subbands in mode 357 are not sequential and are in different "Nyquist Zones"

|     Mode     |         3        |         5        |         7        |
|:------------:|:----------------:|:----------------:|:----------------:|
|   Subbands   | 54, 56, 58...452 | 54, 56, 58...452 | 54, 56, 58...228 |
| Nyquist Zone |         1        |         2        |         3        |

The following code block will create an appropriate frequency array for mode 357 data

In [ ]:
# def sb_to_freq(subband, observation mode):
#     """
#     Converts LOFAR single station subbands to frequency
#     Returns frequency
#     Inputs: subband number, observation mode (3, 5 or 7)
#     """
#     return frequency


In [ ]:
sbs = np.array((np.arange(54,454,2),
                np.arange(54,454,2),
                np.arange(54,230,2)),
                dtype=object) #subbands observed in mode 357
blank_sbs = np.array((np.arange(454,512,2),
                      np.arange(0,54,2),
                      np.arange(454,512,2),
                      np.arange(0,54,2)),
                      dtype=object) #gaps between modes
obs_mode = np.array((3,5,7))
blank_obs_mode = np.array((3,5,5,7))


freqs = np.array([sb_to_freq(sb,mode) for sb,mode in zip(sbs, obs_mode)], dtype=object)
blank_freqs = np.array([sb_to_freq(sb,mode) for sb,mode in zip(blank_sbs, blank_obs_mode)], dtype=object)

sbs=np.concatenate((sbs[0], blank_sbs[0], blank_sbs[1], sbs[1],
                            blank_sbs[2], blank_sbs[3], sbs[2]))

freqs=np.concatenate((freqs[0], blank_freqs[0], blank_freqs[1], freqs[1],
                            blank_freqs[2], blank_freqs[3], freqs[2]))


blank_data = np.zeros((freqs.shape[0],data357.shape[1]))
#1st 200 sbs mode 3, blank, next 200 sbs mode 5, blank, last 88 sbs mode 7
blank_data[:200,:] = data357[:200,:]
blank_len_0 = len(blank_freqs[0]) + len(blank_freqs[1])
blank_data[200 + blank_len_0:400 + blank_len_0,:] = data357[200:400,:]
blank_len_1 = len(blank_freqs[2]) + len(blank_freqs[3])
blank_data[400 + blank_len_0 + blank_len_1 :,:] = data357[400:,:]
data357 = blank_data

mask = np.zeros(data357.shape)
mask[200:256,:] = 1	#1st 200 sbs are mode 3, 56 blank sbs
mask[456:512,:] = 1 #Next 200 sbs are mode 5, 56 blank sbs. Last 88 sbs are mode 7
data357 = np.ma.array(data357, mask=mask)

## Task 5 Remove background

How would you remove the effect of the antenna bandpass filter?


## Task 6 Make a radiospectra spectrogram object

We've seen how easy it is to plot things using radiospectra in previous tutorials but how can we get it to work with I-LOFAR data?

In [ ]:
from radiospectra.spectrogram2 import Spectrogram
from sunpy.net import attrs as a

ilofar_meta = {
    'observatory': 'I-LOFAR',
    'instrument': 'I-LOFAR',
    'detector': 'Mode 357',
    'freqs': frequency_array,
    'times': time_array,
    'wavelength': a.Wavelength(frequency_array[0], frequency_array[-1]),
    'start_time': time_array[0],
    'end_time': time_array[-1]
}
ilofar_spec = Spectrogram(data357, ilofar_meta)

## Task 7 finding times and frequencies of a Type II burst

Notice the Type II burst at ~15:30 UTC in the frequency range of 10-90 MHz. We want to zoom in on this burst and determine 10 points along the fundamental band. Try plotting a zoom in of the burst and use the code block below to save the time and frequencies of each point you click on.
(Hint, try create a new radiospectra Spectrogram with the zoomed in data, the plots update quicker)

In [ ]:
def onclick(event):
    global ix
    ix= event.xdata
    global iy
    iy=event.ydata
    global x_coords
    x_coords.append(ix)
    global y_coords
    y_coords.append(iy)
    plt.plot(ix, iy, 'o', color='r')
    fig.canvas.draw()
    # Disconnect after 10 clicks
    if len(x_coords) == 10:
        fig.canvas.mpl_disconnect(cid)

    return
# find time/frequency to zoom in to e.g. 15:30:00 - 16:30:00 UTC, 10 - 90 MHz
# fig, ax = plt.subplots(figsize=(9,8))
# lofar_zoom_spec.plot()
# x_coords=[]
# y_coords=[]
# cid = fig.canvas.mpl_connect('button_press_event', onclick)
